<a href="https://colab.research.google.com/github/psygrammer/psypy_lm/blob/main/notebooks/ch02/02_fine_tuning_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Fine-Tuning BERT Models

* 싸이그래머 / 싸이파이 [1]
* 김무성

------------

In [1]:
!git clone https://github.com/psygrammer/psypy_lm.git

Cloning into 'psypy_lm'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 42 (delta 11), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [2]:
!ls

psypy_lm  sample_data


In [5]:
# change work_dir
%cd /content/psypy_lm/notebooks/ch02

/content/psypy_lm/notebooks/ch02


In [6]:
ls

02_fine_tuning_bert.ipynb


In [7]:
#@title Activating the GPU
# Main menu->Runtime->Change Runtime Type
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [8]:
 #@title Installing the Hugging Face PyTorch Interface for Bert
!pip install -q transformers

     |████████████████████████████████| 2.0MB 21.1MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
     |████████████████████████████████| 3.2MB 54.6MB/s 


In [11]:
#@title Importing the modules
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler,SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [12]:
from tqdm import tqdm, trange

In [13]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [14]:
#@title Specify CUDA as device for Torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'